In [3]:
import subprocess
import sys

# List of libraries used in your notebook
required_libraries = [
    "gdown",
    "numpy",
    "pandas",
    "requests",
    "scikit-learn",  # sklearn is installed as scikit-learn
    "zipfile36"      # If zipfile isn't included with your Python installation, use zipfile36
]

# Function to install missing libraries
def install_missing_libraries(libraries):
    for lib in libraries:
        try:
            __import__(lib)
        except ImportError:
            print(f"Installing {lib}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])

install_missing_libraries(required_libraries)


Installing scikit-learn...
Installing zipfile36...


In [16]:
import os
import pandas as pd
import gdown
import zipfile

# Function to download and extract zipped folders from Google Drive
def download_and_extract_google_drive_file(drive_file_url, output_directory):
    """
    Downloads a file from Google Drive and extracts it to the specified directory if it is a zip file.
    """
    file_id = drive_file_url.split("/d/")[1].split("/")[0]
    download_url = f"https://drive.google.com/uc?id={file_id}"
    zip_file_name = f"{output_directory}.zip"
    
    print(f"Downloading file from Google Drive: {drive_file_url}")
    gdown.download(download_url, zip_file_name, quiet=False)

    if zipfile.is_zipfile(zip_file_name):
        print(f"Extracting {zip_file_name} to {output_directory}")
        with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
            zip_ref.extractall(output_directory)
        os.remove(zip_file_name)
        print(f"File extracted successfully to {output_directory}")
    else:
        print(f"Downloaded file is not a zip file. Saved as {zip_file_name}")

# Move extracted files to root directory if nested
def normalize_folder_structure(root_folder):
    """
    Moves files from nested subdirectories into the root folder if necessary.
    """
    subdirs = [os.path.join(root_folder, d) for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]
    if len(subdirs) == 1:  # If only one subdirectory exists, move files up
        nested_folder = subdirs[0]
        for file in os.listdir(nested_folder):
            os.rename(os.path.join(nested_folder, file), os.path.join(root_folder, file))
        os.rmdir(nested_folder)  # Remove empty subdirectory
        print(f"Normalized folder structure for {root_folder}")

# Google Drive links for data
weather_drive_url = "https://drive.google.com/file/d/1bkMb3yPXG9Orh-ICbaQ3-mC4OMEw8fqx/view?usp=share_link"
pollen_drive_url = "https://drive.google.com/file/d/1yF0Xn5HfJTEC58pTOCitrjIiVYBhxwXE/view?usp=share_link"

# Directories
weather_folder = "./WeatherTrentinoAltoAdige2024"
pollen_folder = "./PollenTrentinoAltoAdige2024"
output_folder = "./MergedDataTrentinoAltoAdige2024"
os.makedirs(output_folder, exist_ok=True)

# Ensure directories exist or download data
if not os.path.exists(weather_folder):
    download_and_extract_google_drive_file(weather_drive_url, weather_folder)
    normalize_folder_structure(weather_folder)
else:
    print(f"The folder '{weather_folder}' already exists. Skipping download.")

if not os.path.exists(pollen_folder):
    download_and_extract_google_drive_file(pollen_drive_url, pollen_folder)
    normalize_folder_structure(pollen_folder)
else:
    print(f"The folder '{pollen_folder}' already exists. Skipping download.")

# Get list of files
def get_csv_files(folder, suffix):
    """
    Recursively fetches all files with the specified suffix within a folder.
    """
    csv_files = {}
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(suffix):
                base_name = file.replace(suffix, '')
                csv_files[base_name] = os.path.join(root, file)
    return csv_files

# Debug: Check folder contents
print(f"Contents of '{weather_folder}': {os.listdir(weather_folder)}")
print(f"Contents of '{pollen_folder}': {os.listdir(pollen_folder)}")

# Detect files
weather_files = get_csv_files(weather_folder, '_weather.csv')
pollen_files = get_csv_files(pollen_folder, '_pollen.csv')

# Debug: Check discovered files
print(f"Weather files: {weather_files}")
print(f"Pollen files: {pollen_files}")

# Find common municipalities
common_municipalities = set(weather_files.keys()) & set(pollen_files.keys())
print(f"Common municipalities: {common_municipalities}")

# Function to merge files for a municipality
def merge_files(municipality, weather_file, pollen_file, output_file):
    weather_path = weather_file
    pollen_path = pollen_file

    # Load datasets
    weather_data = pd.read_csv(weather_path)
    pollen_data = pd.read_csv(pollen_path)

    # Debug: Check file contents
    print(f"Processing municipality: {municipality}")
    print(f"Weather data shape: {weather_data.shape}")
    print(f"Pollen data shape: {pollen_data.shape}")

    if weather_data.empty or pollen_data.empty:
        print(f"One of the datasets is empty for {municipality}. Skipping.")
        return

    # Parse time column
    weather_data['time'] = pd.to_datetime(weather_data['time'], errors='coerce')
    pollen_data['time'] = pd.to_datetime(pollen_data['time'], errors='coerce')

    # Merge datasets on time
    merged_data = pd.merge(weather_data, pollen_data, on='time', how='inner')
    print(f"Merged data shape for {municipality}: {merged_data.shape}")

    # Save to output file
    merged_data.to_csv(output_file, index=False)

    # Debug: Check if file was saved
    if os.path.exists(output_file):
        print(f"Successfully saved merged file: {output_file}")
    else:
        print(f"Failed to save merged file for {municipality}.")

# Merge files for each municipality
for municipality in common_municipalities:
    weather_file = weather_files[municipality]
    pollen_file = pollen_files[municipality]
    output_file = os.path.join(output_folder, f"{municipality}_data.csv")

    merge_files(municipality, weather_file, pollen_file, output_file)

print("All files processed and merged successfully.")


The folder './WeatherTrentinoAltoAdige2024' already exists. Skipping download.
The folder './PollenTrentinoAltoAdige2024' already exists. Skipping download.
Contents of './WeatherTrentinoAltoAdige2024': ['WeatherTrentinoAltoAdige2024']
Contents of './PollenTrentinoAltoAdige2024': ['PollenTrentinoAltoAdige2024']
Weather files: {'san michele alladige': './WeatherTrentinoAltoAdige2024/WeatherTrentinoAltoAdige2024/san michele alladige_weather.csv', 'castel condino': './WeatherTrentinoAltoAdige2024/WeatherTrentinoAltoAdige2024/castel condino_weather.csv', 'albiano': './WeatherTrentinoAltoAdige2024/WeatherTrentinoAltoAdige2024/albiano_weather.csv', 'gais': './WeatherTrentinoAltoAdige2024/WeatherTrentinoAltoAdige2024/gais_weather.csv', 'sterzing': './WeatherTrentinoAltoAdige2024/WeatherTrentinoAltoAdige2024/sterzing_weather.csv', 'dro': './WeatherTrentinoAltoAdige2024/WeatherTrentinoAltoAdige2024/dro_weather.csv', 'predoi': './WeatherTrentinoAltoAdige2024/WeatherTrentinoAltoAdige2024/predoi_w

In [24]:
import os
import pandas as pd
import gdown
import zipfile

# Function to download and extract zipped folders from Google Drive
def download_and_extract_google_drive_file(drive_file_url, output_directory):
    """
    Downloads a file from Google Drive and extracts it to the specified directory if it is a zip file.
    """
    file_id = drive_file_url.split("/d/")[1].split("/")[0]
    download_url = f"https://drive.google.com/uc?id={file_id}"
    zip_file_name = f"{output_directory}.zip"
    
    print(f"Downloading file from Google Drive: {drive_file_url}")
    gdown.download(download_url, zip_file_name, quiet=False)

    if zipfile.is_zipfile(zip_file_name):
        print(f"Extracting {zip_file_name} to {output_directory}")
        with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
            zip_ref.extractall(output_directory)
        os.remove(zip_file_name)
        print(f"File extracted successfully to {output_directory}")
    else:
        print(f"Downloaded file is not a zip file. Saved as {zip_file_name}")

# Move extracted files to root directory if nested
def normalize_folder_structure(root_folder):
    """
    Moves files from nested subdirectories into the root folder if necessary.
    """
    subdirs = [os.path.join(root_folder, d) for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]
    if len(subdirs) == 1:  # If only one subdirectory exists, move files up
        nested_folder = subdirs[0]
        for file in os.listdir(nested_folder):
            os.rename(os.path.join(nested_folder, file), os.path.join(root_folder, file))
        os.rmdir(nested_folder)  # Remove empty subdirectory
        print(f"Normalized folder structure for {root_folder}")

# Google Drive links for data
weather_drive_url = "https://drive.google.com/file/d/1bkMb3yPXG9Orh-ICbaQ3-mC4OMEw8fqx/view?usp=share_link"
pollen_drive_url = "https://drive.google.com/file/d/1yF0Xn5HfJTEC58pTOCitrjIiVYBhxwXE/view?usp=share_link"

# Directories
weather_folder = "./WeatherTrentinoAltoAdige2024"
pollen_folder = "./PollenTrentinoAltoAdige2024"
output_folder = "./MergedDataTrentinoAltoAdige2024"
os.makedirs(output_folder, exist_ok=True)

# Ensure directories exist or download data
if not os.path.exists(weather_folder):
    download_and_extract_google_drive_file(weather_drive_url, weather_folder)
    normalize_folder_structure(weather_folder)
else:
    print(f"The folder '{weather_folder}' already exists. Skipping download.")

if not os.path.exists(pollen_folder):
    download_and_extract_google_drive_file(pollen_drive_url, pollen_folder)
    normalize_folder_structure(pollen_folder)
else:
    print(f"The folder '{pollen_folder}' already exists. Skipping download.")

# Get list of files
def get_csv_files(folder, suffix):
    """
    Recursively fetches all files with the specified suffix within a folder.
    """
    csv_files = {}
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(suffix):
                base_name = file.replace(suffix, '')
                csv_files[base_name] = os.path.join(root, file)
    return csv_files

# Debug: Check folder contents
print(f"Contents of '{weather_folder}': {os.listdir(weather_folder)}")
print(f"Contents of '{pollen_folder}': {os.listdir(pollen_folder)}")

# Detect files
weather_files = get_csv_files(weather_folder, '_weather.csv')
pollen_files = get_csv_files(pollen_folder, '_pollen.csv')

# Debug: Check discovered files
print(f"Weather files: {weather_files}")
print(f"Pollen files: {pollen_files}")

# Find common municipalities
common_municipalities = set(weather_files.keys()) & set(pollen_files.keys())
print(f"Common municipalities: {common_municipalities}")

# Function to merge files for a municipality
def merge_files(municipality, weather_file, pollen_file, output_file):
    weather_path = weather_file
    pollen_path = pollen_file

    # Load datasets
    weather_data = pd.read_csv(weather_path)
    pollen_data = pd.read_csv(pollen_path)

    # Debug: Check file contents
    print(f"Processing municipality: {municipality}")
    print(f"Weather data shape: {weather_data.shape}")
    print(f"Pollen data shape: {pollen_data.shape}")

    if weather_data.empty or pollen_data.empty:
        print(f"One of the datasets is empty for {municipality}. Skipping.")
        return

    # Parse time column
    weather_data['time'] = pd.to_datetime(weather_data['time'], errors='coerce')
    pollen_data['time'] = pd.to_datetime(pollen_data['time'], errors='coerce')

    # Merge datasets on time
    merged_data = pd.merge(weather_data, pollen_data, on='time', how='inner')
    print(f"Merged data shape for {municipality}: {merged_data.shape}")

    # Save to output file
    merged_data.to_csv(output_file, index=False)

    # Debug: Check if file was saved
    if os.path.exists(output_file):
        print(f"Successfully saved merged file: {output_file}")
    else:
        print(f"Failed to save merged file for {municipality}.")

# Merge files for each municipality
for municipality in common_municipalities:
    weather_file = weather_files[municipality]
    pollen_file = pollen_files[municipality]
    output_file = os.path.join(output_folder, f"{municipality}_data.csv")

    merge_files(municipality, weather_file, pollen_file, output_file)

print("All files processed and merged successfully.")


Downloading...
From (original): https://drive.google.com/uc?id=1bkMb3yPXG9Orh-ICbaQ3-mC4OMEw8fqx
From (redirected): https://drive.google.com/uc?id=1bkMb3yPXG9Orh-ICbaQ3-mC4OMEw8fqx&confirm=t&uuid=5731f4e3-8e8b-4a3b-bcde-90190ef74c2f
To: /Users/davidegiordani/Desktop/BGTUploadData/WeatherTrentinoAltoAdige2024.zip
100%|██████████| 126M/126M [00:02<00:00, 49.5MB/s] 


Extracting ./WeatherTrentinoAltoAdige2024.zip to ./WeatherTrentinoAltoAdige2024
File extracted successfully to ./WeatherTrentinoAltoAdige2024
Normalized folder structure for ./WeatherTrentinoAltoAdige2024


Downloading...
From (original): https://drive.google.com/uc?id=1yF0Xn5HfJTEC58pTOCitrjIiVYBhxwXE
From (redirected): https://drive.google.com/uc?id=1yF0Xn5HfJTEC58pTOCitrjIiVYBhxwXE&confirm=t&uuid=98074a72-aea0-4ef6-a96c-1078b1dc97df
To: /Users/davidegiordani/Desktop/BGTUploadData/PollenTrentinoAltoAdige2024.zip
100%|██████████| 47.7M/47.7M [00:01<00:00, 45.3MB/s]


Extracting ./PollenTrentinoAltoAdige2024.zip to ./PollenTrentinoAltoAdige2024
File extracted successfully to ./PollenTrentinoAltoAdige2024
Normalized folder structure for ./PollenTrentinoAltoAdige2024
Contents of './WeatherTrentinoAltoAdige2024': ['san michele alladige_weather.csv', 'castel condino_weather.csv', 'albiano_weather.csv', 'gais_weather.csv', 'sterzing_weather.csv', 'dro_weather.csv', 'predoi_weather.csv', 'calavino_weather.csv', 'schenna_weather.csv', 'vandoies_weather.csv', 'sanzeno_weather.csv', 'schluderns_weather.csv', 'terre dadige_weather.csv', 'besenello_weather.csv', 'plaus_weather.csv', 'iavre_weather.csv', 'terenten_weather.csv', 'lona-lases_weather.csv', 'unsere liebe frau im walde-st. felix_weather.csv', 'partschins_weather.csv', 'castel ivano_weather.csv', 'ritten_weather.csv', 'barbiano_weather.csv', 'klausen_weather.csv', 'fiera di primiero_weather.csv', 'storo_weather.csv', 'cimone_weather.csv', 'sfruz_weather.csv', 'cunevo_weather.csv', 'franzensfeste_weat

In [25]:
# Directories
merged_data_folder = 'MergedDataTrentinoAltoAdige2024'
daily_data_folder = 'MergedDataTrentinoAltoAdige2024/Daily'
os.makedirs(daily_data_folder, exist_ok=True)

# Function to compute daily averages for a municipality
def compute_daily_averages(file_path, output_path):
    # Load data
    data = pd.read_csv(file_path)

    # Ensure 'time' is parsed correctly
    data['time'] = pd.to_datetime(data['time'], errors='coerce')

    # Add a 'date' column for grouping
    data['date'] = data['time'].dt.date

    # Initialize list for daily data
    daily_records = []

    # Group by date
    for date, group in data.groupby('date'):
        daily_record = {'date': date}

        for column in group.select_dtypes(include=['float64', 'int64']).columns:
            non_nan_count = group[column].notna().sum()
            total_count = len(group[column])

            if non_nan_count >= total_count / 2:
                daily_record[column] = group[column].mean()
            else:
                daily_record[column] = np.nan

        daily_records.append(daily_record)

    # Create DataFrame for daily averages
    daily_data = pd.DataFrame(daily_records)

    # Save the daily-averaged data
    daily_data.to_csv(output_path, index=False)

# Process all municipality files
municipality_files = [f for f in os.listdir(merged_data_folder) if f.endswith('_data.csv')]

for municipality_file in municipality_files:
    file_path = os.path.join(merged_data_folder, municipality_file)
    output_path = os.path.join(daily_data_folder, municipality_file)

    compute_daily_averages(file_path, output_path)

print("Daily averages computed and saved for all municipalities.")

Daily averages computed and saved for all municipalities.


In [1]:
import os
import pandas as pd
import requests

# Directories
daily_data_folder = './MergedDataTrentinoAltoAdige2024/Daily'
valleys_data_folder = './MergedDataTrentinoAltoAdige2024/Valleys'
os.makedirs(valleys_data_folder, exist_ok=True)

# GitHub raw file URL for the mapping file
mapping_github_url = "https://raw.githubusercontent.com/mattia-rampazzo/bdt_project/ea35c55d30c7b36638e67791edd6a1339d700045/Predictions/Merged_Valleys_and_Municipalities.csv"
local_mapping_file = './Merged_Valleys_and_Municipalities.csv'

# Function to download the mapping file from GitHub
def fetch_mapping_file(github_url, local_path):
    response = requests.get(github_url)
    if response.status_code == 200:
        with open(local_path, 'w') as file:
            file.write(response.text)
        print(f"Mapping file downloaded and saved to {local_path}")
    else:
        raise Exception(f"Failed to download mapping file. Status code: {response.status_code}")

# Download the mapping file
fetch_mapping_file(mapping_github_url, local_mapping_file)

# Load valley-municipality mapping
mapping_data = pd.read_csv(local_mapping_file)

# Create a dictionary mapping municipalities to valleys
municipality_to_valley = mapping_data.set_index('normalized_municipality')['valley'].to_dict()

# Process all municipality files
municipality_files = [f for f in os.listdir(daily_data_folder) if f.endswith('_data.csv')]

for municipality_file in municipality_files:
    municipality_name = municipality_file.replace('_data.csv', '')
    valley_name = municipality_to_valley.get(municipality_name)

    if valley_name:
        # Create valley folder if it doesn't exist
        valley_folder = os.path.join(valleys_data_folder, valley_name)
        os.makedirs(valley_folder, exist_ok=True)

        # Move municipality file to the valley folder
        source_path = os.path.join(daily_data_folder, municipality_file)
        destination_path = os.path.join(valley_folder, municipality_file)
        os.rename(source_path, destination_path)

print("Municipality files organized into valley folders.")


Mapping file downloaded and saved to ./Merged_Valleys_and_Municipalities.csv
Municipality files organized into valley folders.


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

# Directories
valleys_data_folder = 'MergedDataTrentinoAltoAdige2024/Valleys'
valleys_models_folder = 'ValleysModels1'
os.makedirs(valleys_models_folder, exist_ok=True)

# Pollen columns to predict
pollen_columns = [
    'Alder pollen (grains/m³)', 'Birch pollen (grains/m³)', 
    'Grass pollen (grains/m³)', 'Mugwort pollen (grains/m³)', 
    'Olive pollen (grains/m³)', 'Ragweed pollen (grains/m³)'
]

# Function to preprocess and train a unified model
def train_valley_model(valley_folder, output_folder, valley_name):
    # Merge all municipality data
    merged_data = pd.DataFrame()
    municipality_files = [f for f in os.listdir(valley_folder) if f.endswith('_data.csv')]

    for municipality_file in municipality_files:
        file_path = os.path.join(valley_folder, municipality_file)
        data = pd.read_csv(file_path)
        data['municipality'] = municipality_file.replace('_data.csv', '')
        merged_data = pd.concat([merged_data, data], ignore_index=True)

    # Ensure 'date' column is in datetime format
    merged_data['date'] = pd.to_datetime(merged_data['date'], errors='coerce')

    # Initialize results
    results = {}
    models = {}

    # Iterate over each pollen type
    for pollen_type in pollen_columns:
        if pollen_type not in merged_data.columns:
            print(f"Warning: {pollen_type} is missing in the dataset. Skipping...")
            continue

        # Fill missing pollen values
        merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')

        # Create lagged features
        merged_data[f'{pollen_type}_lag_1'] = merged_data[pollen_type].shift(1)
        merged_data[f'{pollen_type}_lag_2'] = merged_data[pollen_type].shift(2)
        merged_data[f'{pollen_type}_lag_3'] = merged_data[pollen_type].shift(3)

        # Drop rows with NaN values due to lagging
        data_cleaned = merged_data.dropna()

        # Define features and target
        pollen_lagged_features = [f'{pollen_type}_lag_1', f'{pollen_type}_lag_2', f'{pollen_type}_lag_3']
        weather_features = [col for col in merged_data.columns if col.startswith('temperature') or col.startswith('precipitation')]
        X = data_cleaned[pollen_lagged_features + weather_features]
        y = data_cleaned[pollen_type]

        # Split data into train-test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train Random Forest model
        rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=None)
        rf.fit(X_train, y_train)

        # Evaluate the model
        y_pred = rf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Save results and model
        results[pollen_type] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
        models[pollen_type] = rf

        print(f"{pollen_type} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}")

    # Save results to CSV
    results_path = os.path.join(output_folder, f'{valley_name}_results.csv')
    pd.DataFrame(results).T.to_csv(results_path)

    # Save models to a pickle file with valley name
    models_path = os.path.join(output_folder, f'{valley_name}_models.pkl')
    with open(models_path, 'wb') as f:
        pickle.dump(models, f)

    # Copy models to a central folder
    central_model_path = os.path.join(valleys_models_folder, f'{valley_name}_models.pkl')
    with open(central_model_path, 'wb') as f:
        pickle.dump(models, f)

    print(f"Models and results saved for valley: {valley_name} and copied to ValleysModels")

# Process all valleys
for valley_name in os.listdir(valleys_data_folder):
    valley_folder = os.path.join(valleys_data_folder, valley_name)
    if os.path.isdir(valley_folder):
        print(f"Processing valley: {valley_name}")
        output_model_folder = os.path.join(valley_folder, 'Models')
        os.makedirs(output_model_folder, exist_ok=True)
        train_valley_model(valley_folder, output_model_folder, valley_name)

Processing valley: Val Venosta


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_37383/2999670981.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Alder pollen (grains/m³) - RMSE: 0.25, MAE: 0.05, R2: 0.99


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_37383/2999670981.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Birch pollen (grains/m³) - RMSE: 2.72, MAE: 0.93, R2: 0.89


...

Grass pollen (grains/m³) - RMSE: 2.13, MAE: 0.73, R2: 0.94
Mugwort pollen (grains/m³) - RMSE: 0.00, MAE: 0.00, R2: 1.00


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_37383/2999670981.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')
/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_37383/2999670981.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Olive pollen (grains/m³) - RMSE: 0.13, MAE: 0.02, R2: 0.88


/var/folders/q2/9lh55t_s1k56_qhpfd2m00f80000gn/T/ipykernel_37383/2999670981.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[pollen_type] = merged_data[pollen_type].interpolate().fillna(method='bfill').fillna(method='ffill')


Ragweed pollen (grains/m³) - RMSE: 0.14, MAE: 0.02, R2: 0.87
Models and results saved for valley: Valle di Rendena and copied to ValleysModels
